In [1]:
import kmr

In [2]:
seq = [(1, 2), (3, 1), (2, 2), (1, 1), (2, 3), (1, 2)]

kmr.pohl_sort_rename(seq)

([1, 4, 2, 0, 3, 1], {0: 3, 1: 0, 2: 2, 3: 4, 4: 1})

In [3]:
text = "abaabbaa"
names, entries = kmr.pohl_kmr(text)

print('names')
for k, v in names.items():
    print(k, [e + 1 for e in v[:len(text)]])

names
1 [1, 2, 1, 1, 2, 2, 1, 1]
2 [2, 4, 1, 2, 5, 4, 1, 3]
4 [3, 6, 1, 4, 8, 7, 2, 5]


In [4]:
kmr.pohl_sort_rename(list('abaabbaa'))

([0, 1, 0, 0, 1, 1, 0, 0], {0: 0, 1: 1})

In [5]:
s = [(0, 1), (1, 2), (1, 2), (0, 1), (2, 1)]
kmr.counting_sort(s, 1)

[(0, 1), (0, 1), (2, 1), (1, 2), (1, 2)]

In [6]:
kmr.radix_sort(s, 2)

[(0, 1), (0, 1), (1, 2), (1, 2), (2, 1)]

In [7]:
list(kmr.basic_search('ab', 'ababab'))

KeyError: 2